In [5]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# Load the unclean dataset
df_unclean = pd.read_csv('customer_purchase_data_unclean.csv')

In [7]:
df_unclean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Customer_ID         1992 non-null   object 
 1   Age                 1994 non-null   float64
 2   Gender              1993 non-null   float64
 3   Region              1994 non-null   float64
 4   Purchase_Category   1997 non-null   float64
 5   Total_Amount_Spent  1995 non-null   float64
 6   Discount_Used       1993 non-null   float64
 7   High_Spender        1996 non-null   float64
dtypes: float64(7), object(1)
memory usage: 131.4+ KB


In [9]:
df_unclean.isnull().sum()

Customer_ID           108
Age                   106
Gender                107
Region                106
Purchase_Category     103
Total_Amount_Spent    105
Discount_Used         107
High_Spender          104
dtype: int64

In [10]:
# 1. Remove duplicate rows
df_cleaned = df_unclean.drop_duplicates()

# 2. Handle missing values (impute with mean for numeric and mode for categorical)
numeric_cols = df_cleaned.select_dtypes(include=['number']).columns
categorical_cols = df_cleaned.select_dtypes(exclude=['number']).columns

num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

df_cleaned[numeric_cols] = num_imputer.fit_transform(df_cleaned[numeric_cols])
df_cleaned[categorical_cols] = cat_imputer.fit_transform(df_cleaned[categorical_cols])




C:\Users\kusha\AppData\Local\Temp\ipykernel_7560\4265057798.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[numeric_cols] = num_imputer.fit_transform(df_cleaned[numeric_cols])
C:\Users\kusha\AppData\Local\Temp\ipykernel_7560\4265057798.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[categorical_cols] = cat_imputer.fit_transform(df_cleaned[categorical_cols])


In [11]:
# 3. Remove outliers using Z-score method
from scipy.stats import zscore
z_scores = np.abs(zscore(df_cleaned[numeric_cols]))
df_cleaned = df_cleaned[(z_scores < 3).all(axis=1)]



In [13]:
# Save the cleaned dataset
df_cleaned.to_csv('customer_purchase_data_cleaned_final.csv', index=False)

print("Data cleaning complete! Cleaned dataset saved.")

Data cleaning complete! Cleaned dataset saved.


In [14]:
df_cleaned.isnull().sum()

Customer_ID           0
Age                   0
Gender                0
Region                0
Purchase_Category     0
Total_Amount_Spent    0
Discount_Used         0
High_Spender          0
dtype: int64

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv('customer_purchase_data_cleaned_final.csv')

# Encode categorical variables
label_encoders = {}
categorical_cols = ['Gender', 'Region', 'Purchase_Category', 'Discount_Used', 'High_Spender']
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Define features and target
X = df.drop(columns=['Customer_ID', 'High_Spender'])
y = df['High_Spender']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression model
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)

# Make predictions
y_pred = log_reg.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Logistic Regression Accuracy: {accuracy * 100:.2f}%')


Logistic Regression Accuracy: 67.39%


c:\Users\kusha\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
